In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train_data = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv')
train_data.head()

In [ ]:
y = train_data['Survived']
train_data = train_data.drop('Survived',axis=1)


In [ ]:
train_data.shape

In [ ]:
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')
test_data

In [ ]:
#append train and test data to form total data
data = pd.concat([train_data,test_data])
data.tail()

In [ ]:
data.describe()

In [ ]:
import seaborn as sns
sns.histplot(y)

In [ ]:
nulls = data.isna().sum()
nulls

In [ ]:
data.info()

In [ ]:
#Drop the passenger id, name column
data = data.drop(['Name','PassengerId','Ticket','Cabin'],axis =1)
data.shape

In [ ]:
data['Sex'].value_counts()
data['Sex'] = data['Sex'].replace('male',1).replace('female',0)
data.head()

In [ ]:
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Fare'] = data['Fare'].fillna(data['Fare'].median())
data.head()

In [ ]:
data = pd.get_dummies(data, columns=['Embarked'])

In [ ]:
data.shape

In [ ]:
#Split your dataset back into train and test
train = data.iloc[:100000].values
train.shape

In [ ]:
sns.pairplot(data)

In [ ]:
test = data.iloc[100000:].values
test.shape

In [ ]:
import lightgbm
lgb_train = lightgbm.Dataset(train, label=y )

In [ ]:

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

model = lightgbm.train(parameters,
                       lgb_train,
                        100)


In [ ]:
pred = model.predict(test)
pred

In [ ]:
for i in range(0,len(test)):
    if pred[i]>=.5:
        
        pred[i]=1
    else:  
        pred[i]=0
pred

In [ ]:
#Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
ext = ExtraTreesClassifier()
ext.fit(train,y)

In [ ]:
ext_pred = ext.predict(test)
ext_pred

In [ ]:
#Extra Trees Classifier model submission
ext_submission = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': ext_pred})
ext_submission.to_csv('ext_submission.csv', index=False)

In [ ]:
#randome forest model training
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5, random_state=0,n_estimators = 500,criterion='entropy')
clf.fit(train, y)

In [ ]:
rf_pred = clf.predict(test)
rf_pred

In [ ]:
#making a submission
submission = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv')
submission

In [ ]:
my_submission = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': pred})
my_submission.to_csv('submission.csv', index=False)

In [ ]:
#random forest model submission
rf_submission = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': rf_pred})
rf_submission.to_csv('rf_submission.csv', index=False)

In [ ]:
s = pd.read_csv('ext_submission.csv')
s